<a href="https://colab.research.google.com/github/scyllastrikes/WildWestH4CKED4/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Flight Delay Prediction Using Machine Learning**

### **Project Overview**

This project aims to predict flight arrival delays using historical flight data and various influential factors such as weather conditions, crew availability, and traffic congestion. Accurate predictions of flight delays can help improve passenger experience and airline operational efficiency by enabling proactive measures to mitigate delays.

### **Objectives**

- Build a machine learning model to predict flight delays based on factors such as departure time, weather, traffic, and crew issues.
- Use **PyTorch** to create a neural network model that processes the data and learns patterns in flight delays.
- Evaluate the model’s performance using the **Mean Squared Error (MSE)** and other relevant metrics.




In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:

# Load the dataset
df = pd.read_csv('/airplane_flights_dataset.csv')

# Convert time columns to datetime
df['Departure Time'] = pd.to_datetime(df['Departure Time'])
df['Arrival Time'] = pd.to_datetime(df['Arrival Time'])
df['Expected Time'] = pd.to_datetime(df['Expected Time'])

# Calculate delay in minutes
df['Delay in Minutes'] = (df['Arrival Time'] - df['Expected Time']).dt.total_seconds() / 60

# Extract useful features
df['Departure Hour'] = df['Departure Time'].dt.hour
df['Departure Weekday'] = df['Departure Time'].dt.weekday

# Convert bool columns to ints
df['Peak Hour'] = df['Peak Hour'].astype(int)
df['Bad Weather'] = df['Bad Weather'].astype(int)
df['Traffic Congestion'] = df['Traffic Congestion'].astype(int)
df['Crew Availability Issue'] = df['Crew Availability Issue'].astype(int)

# Define features (X) and target (y)
X = df[['Departure Hour', 'Departure Weekday', 'Peak Hour', 'Bad Weather', 'Traffic Congestion', 'Crew Availability Issue']].values
y = df['Delay in Minutes'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)


In [ ]:

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(6, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()


In [ ]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer


In [ ]:

epochs = 140
for epoch in range(epochs):
    model.train()

    # Forward pass
    y_pred_train = model(X_train_tensor)
    loss = criterion(y_pred_train, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [10/140], Loss: 2600.9895
Epoch [20/140], Loss: 2589.1282
Epoch [30/140], Loss: 2576.5056
Epoch [40/140], Loss: 2563.0293
Epoch [50/140], Loss: 2548.6143
Epoch [60/140], Loss: 2533.1816
Epoch [70/140], Loss: 2516.6743
Epoch [80/140], Loss: 2499.0627
Epoch [90/140], Loss: 2480.2927
Epoch [100/140], Loss: 2460.3496
Epoch [110/140], Loss: 2439.2107
Epoch [120/140], Loss: 2416.8279
Epoch [130/140], Loss: 2393.1770
Epoch [140/140], Loss: 2368.2605


In [ ]:
# Step 6: Evaluate the model on the test set

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    test_loss = criterion(y_pred_test, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 2365.8608


In [ ]:
# Step 7: Making a new prediction

# Example new flight feature vector (scaled like before)
new_flight = scaler.transform([[10, 3, 1, 0, 1, 0]])  # Example: 10AM, Wednesday, peak hour, good weather, traffic, no crew issue
new_flight_tensor = torch.tensor(new_flight, dtype=torch.float32)

# Predict the delay
model.eval()
with torch.no_grad():
    predicted_delay = model(new_flight_tensor)
    print(f'Predicted delay: {predicted_delay.item():.2f} minutes')


Predicted delay: 4.01 minutes
